# Personalized Learning Demo - Quickstart

A full-stack A2UI sample demonstrating personalized educational content generation.

**Contributed by Google Public Sector's Rapid Innovation Team.**

![Personalized Learning Demo](assets/hero.png)

---

## The Scenario

**Maria Thompson** is a pre-med student at Cymbal University preparing for the MCAT. She excels in biology (92%) but struggles with chemistry concepts—particularly the common misconception that "energy is stored in ATP bonds."

How we find that information about a student's misconceptions across multiple courses is actually part of a broader project at Google Public Sector. But for now, we're just running with information we have related to one of those misconceptions, represented by the text files in the `learner_context/` directory.

This demo includes a [learner profile visualization](http://localhost:5174/maria-context.html) showing Maria's:
- Academic background and current proficiency levels
- Identified misconceptions to address
- Learning preferences (visual-kinesthetic, sports/gym analogies)

This profile represents the kind of data a real personalization pipeline would generate from learning management systems, assessment results, and curriculum graphs. Once we have that data, how do we best use it to impact a student's learning trajectory?

We think the A2UI framework enables an excellent learning experience, and this demo intends to show how.

---

## How Content Is Generated

**Content Source:** [OpenStax](https://openstax.org/) — free, peer-reviewed textbooks covering 167 chapters across biology, chemistry, physics, and more.

**Generation Pipeline:**
- User requests a topic (e.g., "Help me understand ATP")
- The agent uses an LLM to match the topic to the most relevant OpenStax chapter
- Content is fetched and transformed into A2UI components (flashcards, quizzes)
- The frontend renders whatever A2UI JSON the agent returns

**Learn More:**
- [How A2UI Works](http://localhost:5174/a2ui-primer.html) — interactive explanation in the demo
- [A2UI Specification](../../docs/) — canonical documentation in this repo

---

## What You'll Learn

| Concept | What This Demo Shows |
|---------|---------------------|
| **Remote Agent Deployment** | Deploy an AI agent to Vertex AI Agent Engine that runs independently from your UI |
| **A2A Protocol** | Use the Agent-to-Agent protocol to communicate between your frontend and the remote agent |
| **Custom UI Components** | Extend A2UI with custom components (Flashcard, QuizCard) beyond the standard library |
| **Dynamic Content Generation** | Generate personalized A2UI JSON on-the-fly based on user requests |
| **Intelligent Content Matching** | Use LLMs to match user topics to relevant textbook content (167 OpenStax chapters) |

---

## Architecture

![Architecture Diagram](assets/architecture.jpg)

In production agentic systems:
- **Agents run remotely** — they scale independently, can be updated without redeploying the UI, and may be operated by third parties
- **UI is decoupled** — the frontend renders whatever A2UI JSON the agent sends, without knowing the agent's implementation
- **A2A enables interoperability** — any A2A-compatible agent can power your UI, regardless of how it's built

---

## How This Notebook Is Organized

| Section | What It Does |
|---------|--------------|
| **Step 1: Environment Setup** | Creates Python virtual environment and installs all dependencies |
| **Step 2: Configuration** | Sets your GCP project ID |
| **Step 3: GCP Authentication** | Authenticates with Google Cloud and enables required APIs |
| **Step 4: Deploy Agent** | Deploys the AI agent to Vertex AI Agent Engine |
| **Step 5: Configure & Run** | Creates config files and launches the demo |
| **Step 6 (Optional)** | Generate audio/video content with NotebookLM |
| **Appendix: Local Development** | Run entirely locally without cloud deployment |

---

## Prerequisites

- **Node.js 18+** — [Download](https://nodejs.org/)
- **Python 3.11+** — [Download](https://www.python.org/downloads/)
- **Google Cloud project with billing enabled** — [Console](https://console.cloud.google.com/)
- **gcloud CLI installed** — [Install Guide](https://cloud.google.com/sdk/docs/install)

## Imports

Run this cell first to load all Python modules used throughout the notebook.

In [ ]:
import subprocess
import sys
import os

## Step 1: Environment Setup

First, we'll create an isolated Python environment and install all dependencies. This ensures the demo doesn't conflict with other Python projects on your system.

### 1a. Create Python Virtual Environment

In [ ]:
# Create virtual environment if it doesn't exist
venv_path = os.path.join(os.getcwd(), ".venv")
if not os.path.exists(venv_path):
    print("Creating Python virtual environment...")
    subprocess.run([sys.executable, "-m", "venv", ".venv"], check=True)
    print(f"✅ Created virtual environment at {venv_path}")
else:
    print(f"✅ Virtual environment already exists at {venv_path}")

print("\n⚠️  IMPORTANT: Restart your Jupyter kernel to use the new environment!")
print("   In VS Code: Click the kernel selector (top right) → Select '.venv'")
print("   In JupyterLab: Kernel → Change Kernel → Python (.venv)")

### 1b. Install Python Dependencies

After selecting the `.venv` kernel, run this cell to install all required Python packages.

**Note:** We explicitly use `https://pypi.org/simple/` to ensure packages come from the official Python Package Index, avoiding issues with corporate proxies or custom registries.

In [ ]:
# Install Python dependencies using the canonical PyPI index
print("Installing Python dependencies from PyPI...")
packages = [
    "google-adk>=0.3.0",
    "google-genai>=1.0.0",
    "google-cloud-storage>=2.10.0",
    "python-dotenv>=1.0.0",
    "litellm>=1.0.0",
    "vertexai",
]

subprocess.run([
    sys.executable, "-m", "pip", "install", "-q",
    "--index-url", "https://pypi.org/simple/",
    "--trusted-host", "pypi.org",
    "--trusted-host", "files.pythonhosted.org",
    *packages
], check=True)

print("✅ Python dependencies installed")

### 1c. Install Node.js Dependencies

Now we'll install the frontend dependencies. This includes the A2UI renderer library and the demo's own packages.

In [ ]:
# Build the A2UI Lit renderer (using public npm registry)
print("Building A2UI Lit renderer...")
subprocess.run(
    "npm install --registry https://registry.npmjs.org/ && npm run build",
    shell=True,
    cwd="../../renderers/lit",
    check=True
)
print("✅ A2UI renderer built")

# Install demo dependencies
print("\nInstalling demo dependencies...")
subprocess.run(
    "npm install --registry https://registry.npmjs.org/",
    shell=True,
    check=True
)
print("✅ Demo dependencies installed")

## Step 2: Configuration

Set your Google Cloud project ID below. This is the project where the agent will be deployed.

In [ ]:
PROJECT_ID = "your-project-id"  # <-- CHANGE THIS to your GCP project ID
LOCATION = "us-central1"

## Step 3: GCP Authentication & API Setup

Authenticate with Google Cloud and enable the required APIs. This will open browser windows for authentication.

In [ ]:
# Authenticate with Google Cloud
!gcloud auth login
!gcloud config set project {PROJECT_ID}
!gcloud auth application-default login

In [ ]:
# Enable required APIs
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable storage.googleapis.com
!gcloud services enable cloudresourcemanager.googleapis.com

# Create staging bucket for Agent Engine (if it doesn't exist)
!gsutil mb -l {LOCATION} gs://{PROJECT_ID}_cloudbuild 2>/dev/null || echo "Bucket already exists"

print("\n✅ GCP APIs enabled and staging bucket ready")

## Step 4: Deploy the A2UI Agent

The agent generates personalized learning content and runs on Vertex AI Agent Engine. Deployment takes 2-5 minutes.

**Why deploy remotely?** A2UI is designed for remote agents - your UI runs in the browser while the agent runs on a server. This mirrors real-world architectures where agents scale independently and may even be operated by third parties.

In [ ]:
# Deploy the agent to Vertex AI Agent Engine (takes 2-5 minutes)
print("Deploying agent to Vertex AI Agent Engine...")
print("This takes 2-5 minutes. Watch for the Resource ID at the end.\n")

result = subprocess.run(
    [sys.executable, "deploy.py", "--project", PROJECT_ID, "--location", LOCATION],
    cwd="agent"
)

if result.returncode != 0:
    print("\n❌ Deployment failed. Check the error messages above.")
else:
    print("\n✅ Deployment complete! Copy the Resource ID from the output above.")

## Step 5: Configure & Run

Fill in the Resource ID from the deployment output above, then create the configuration file.

In [ ]:
# Get your project NUMBER (different from project ID)
result = subprocess.run(
    ["gcloud", "projects", "describe", PROJECT_ID, "--format=value(projectNumber)"], 
    capture_output=True, text=True
)
PROJECT_NUMBER = result.stdout.strip()
print(f"Project Number: {PROJECT_NUMBER}")

In [ ]:
# Paste the Resource ID from the deployment output in Step 4
AGENT_RESOURCE_ID = "YOUR_RESOURCE_ID_HERE"  # <-- PASTE YOUR RESOURCE ID HERE

In [ ]:
# Create .env file
env_content = f"""# Generated by Quickstart.ipynb
GOOGLE_CLOUD_PROJECT={PROJECT_ID}
AGENT_ENGINE_PROJECT_NUMBER={PROJECT_NUMBER}
AGENT_ENGINE_RESOURCE_ID={AGENT_RESOURCE_ID}
"""

with open(".env", "w") as f:
    f.write(env_content)

print("Created .env file:")
print(env_content)
print("✅ Configuration complete!")

### Run the Demo

Everything is set up! Run these commands in your terminal (not in the notebook):

```bash
cd samples/personalized_learning
npm run dev
```

Then open **http://localhost:5174**

### Try These Prompts

| Prompt | What Happens |
|--------|-------------|
| "Help me understand ATP" | Generates flashcards from OpenStax |
| "Quiz me on bond energy" | Interactive quiz cards |
| "Play the podcast" | Audio player (requires Step 6) |
| "Show me a video" | Video player (requires Step 6) |

---

## Step 6 (Optional): Generate Audio & Video with NotebookLM

The demo includes audio and video players, but you need to generate the media files. NotebookLM can create personalized podcasts based on the learner context.

### Prerequisites

- A Google account with access to [NotebookLM](https://notebooklm.google.com/)
- The `learner_context/` files from this demo

---

### Part A: Generate a Personalized Podcast

**1. Create a NotebookLM Notebook**

Go to [notebooklm.google.com](https://notebooklm.google.com/) and create a new notebook.

**2. Upload Learner Context Files**

Upload all files from the `learner_context/` directory:
- `01_maria_learner_profile.txt` - Maria's background and learning preferences  
- `02_chemistry_bond_energy.txt` - Bond energy concepts
- `03_chemistry_thermodynamics.txt` - Thermodynamics content
- `04_biology_atp_cellular_respiration.txt` - ATP and cellular respiration
- `05_misconception_resolution.txt` - Common misconceptions to address
- `06_mcat_practice_concepts.txt` - MCAT-focused content

These files give NotebookLM the context to generate personalized content.

**3. Generate the Audio Overview**

- Click **Notebook guide** in the right sidebar
- Click **Audio Overview** → **Generate**
- Wait for generation to complete (typically 2-5 minutes)
- NotebookLM will create a podcast-style discussion about the uploaded content

**4. Customize the Audio (Optional)**

Before generating, you can click **Customize** to provide specific instructions:

```
Create a podcast for Maria, a pre-med student preparing for the MCAT. 
Use gym and fitness analogies since she loves working out.
Focus on explaining why "energy stored in bonds" is a misconception.
Make it conversational and engaging, about 5-7 minutes long.
```

**5. Download and Install the Podcast**

- Once generated, click the **⋮** menu on the audio player
- Select **Download**
- Save the file as `podcast.m4a`
- Copy to the demo's assets directory:

In [ ]:
# Copy your downloaded podcast to the assets directory
# Replace ~/Downloads/podcast.m4a with your actual download path
!cp ~/Downloads/podcast.m4a public/assets/podcast.m4a

# Verify the file was copied
!ls -la public/assets/

---

### Part B: Create a Video

In NotebookLM with your learner context loaded:
- In the **studio** tab, click "Video Overview"
- This creates a video file you can view and export

Export your video as MP4 and copy to the assets directory:

In [ ]:
# Copy your video to the assets directory
# Replace ~/Downloads/demo.mp4 with your actual file path
!cp ~/Downloads/demo.mp4 public/assets/demo.mp4

# Verify both media files are in place
!ls -la public/assets/

#### Option 2: Use Placeholder/Stock Content

For demo purposes, you can use any MP4 video file. Rename it to `demo.mp4` and place it in `public/assets/`.

---

### Verify Media Files

After copying your files, verify they're accessible:

In [ ]:
print("Media files status:")
print("-" * 40)

podcast_path = "public/assets/podcast.m4a"
video_path = "public/assets/demo.mp4"

if os.path.exists(podcast_path):
    size_mb = os.path.getsize(podcast_path) / (1024 * 1024)
    print(f"✅ Podcast: {podcast_path} ({size_mb:.1f} MB)")
else:
    print(f"❌ Podcast: {podcast_path} NOT FOUND")
    
if os.path.exists(video_path):
    size_mb = os.path.getsize(video_path) / (1024 * 1024)
    print(f"✅ Video: {video_path} ({size_mb:.1f} MB)")
else:
    print(f"❌ Video: {video_path} NOT FOUND")

print("-" * 40)
print("\nRun 'npm run dev' and try:")
print('  • "Play the podcast" - to hear the audio')
print('  • "Show me a video" - to watch the video')

---

## Content Attribution

### OpenStax

Educational content is sourced from [OpenStax](https://openstax.org/), licensed under [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/).

Specifically: [Biology for AP® Courses](https://openstax.org/details/books/biology-ap-courses) - OpenStax, Rice University

---

## Security Notice

> **Warning:** When building production applications, treat any agent outside your control as potentially untrusted. This demo connects to Agent Engine within your own GCP project. Always review agent code before deploying.

---

## Limitations & Known Issues

This is a demonstration, not a production system. Here's what can/will break:

| What You Try | What Happens | Why |
|--------------|--------------|-----|
| **Ask for study materials across multiple topics at once** | Retrieval returns wrong content | The agent is designed to match to a single OpenStax chapter; multi-topic queries need more sophisticated retrieval. (There are many good ways to do this.) |
| **"Play podcast about X"** | Nothing plays (or wrong content) | Audio is pre-generated via NotebookLM, not dynamically created |
| **Sidebar navigation, settings, etc.** | Nothing happens | The UI is styled to resemble a Google product, but only the chat functionality is implemented |

### What This Demo Is (and Isn't)

**Is:** A working example of A2UI's architecture—remote agent deployment, A2A protocol, custom components, and dynamic content generation.

**Isn't:** A complete learning platform. The personalization pipeline, multi-topic retrieval, and non-chat UI elements are placeholders demonstrating where real implementations would go.